In [1]:
import numpy as np
from scipy.optimize import least_squares
#from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
#
# if pade.py is not in the current directory, set this path:
#
#import sys
#sys.path.append('../Python_libs')
from rac_aux import *

In [2]:
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068
#
#  files in the current directory do not need the path name
#
#df = pd.read_csv("/home/thomas/Python/StabPlots/Stab_data/1D_a0.2_b0_c0.14/crossing_1.dat", delim_whitespace=True)
df = pd.read_csv("sb_rac.csv")
#df = pd.read_csv("crossing_1.dat", delim_whitespace=True)

plot_it=False
if plot_it:
    plt.cla()
    plt.plot(df.l.values, df.E1.values, 'o-')
    plt.plot(df.l.values, df.E2.values, 'o-')
    plt.plot(df.l.values, df.E3.values, 'o-')
    plt.show()
df[:5]

,l,E1,E2,E3,E4
0,0.0,0.469219,1.365173,1.938043,3.271038
1,0.2,0.457846,1.314401,1.844238,3.242423
2,0.4,0.446073,1.251127,1.760926,3.215267
3,0.6,0.433754,1.173608,1.690292,3.189317
4,0.8,0.420648,1.082215,1.632448,3.164365


In [3]:
i_neg = np.argmin(abs(df.E1.values))
if df.E1[i_neg] > 0:
    i_neg += 1
ls = df.l.values[i_neg:]
print('N=',len(ls))
Es = df.E1.values[i_neg:]
if plot_it:
    plt.cla()
    plt.plot(df.l.values, df.E1.values, 'b-')
    plt.plot(df.l.values, df.E2.values, 'b-')
    plt.plot(df.l.values, df.E3.values, 'b-')
    plt.plot(ls, Es, 'o', color="orange")
    plt.show()

N= 89


In [4]:
k2s = -Es
ks  = np.sqrt(k2s)

In [9]:
# start params
p31_opt = [2.4022, 0.2713, 1.2813, 0.4543]
p42_opt = [2.3919, 0.2964, 1.3187, 1.3736,  0.29655, 0.5078]
p0s = p42_opt[0:5] + p31_opt[3:] + p42_opt[5:] + [1]
print(p0s)
print(chi2_gen(p0s, ks, k2s, ls, pade_53_lsq))

[2.3919, 0.2964, 1.3187, 1.3736, 0.29655, 0.4543, 0.5078, 1]
33.16551690973274


In [6]:
#
#  the derivative of [5,3] works
#

N=6

df1s = pade_gen_j_lsq(p0s, ks[-N:], k2s[-N:], ls[-N:], pade_53_lsq)
print("num grad:\n", df1s)

df2s = pade_53j_lsq(p0s, ks[-N:], k2s[-N:], ls[-N:])
print("ana grad:\n", df2s)

np.sqrt(np.sum(np.square(df1s-df2s)))

num grad:
 [[ 0.05109042  0.03439393  0.02001603  0.13483982  0.10638061  0.23072723
  -0.00033939 -0.02434429]
 [ 0.05115288  0.03427232  0.01982363  0.13499377  0.10717097  0.23184297
  -0.00033769 -0.0243752 ]
 [ 0.05121644  0.03415321  0.01963566  0.1351494   0.10796052  0.23295666
  -0.00033602 -0.02440661]
 [ 0.05128105  0.03403652  0.01945195  0.13530663  0.10874926  0.23406828
  -0.0003344  -0.0244385 ]
 [ 0.05134669  0.03392217  0.01927238  0.13546539  0.1095372   0.23517783
  -0.00033281 -0.02447086]
 [ 0.05141329  0.0338101   0.01909678  0.13562559  0.11032434  0.23628531
  -0.00033126 -0.02450365]]
ana grad:
 [[ 0.05109042  0.03439393  0.02001603  0.13483982  0.10638061  0.23072723
  -0.00033939 -0.02434429]
 [ 0.05115288  0.03427232  0.01982363  0.13499377  0.10717097  0.23184297
  -0.00033769 -0.0243752 ]
 [ 0.05121644  0.03415321  0.01963566  0.1351494   0.10796052  0.23295666
  -0.00033602 -0.02440661]
 [ 0.05128105  0.03403652  0.01945195  0.13530663  0.10874926  0.234

1.2168254197294816e-09

In [ ]:
# start params
p31_opt = [2.4022, 0.2713, 1.2813, 0.4543]
p42_opt = [2.3919, 0.2964, 1.3187, 1.3736,  0.29655, 0.5078]
p0s = p42_opt[0:5] + p31_opt[3:] + p42_opt[5:] + [1]
print(p0s)

In [15]:
#
#  test results of other minimizers wrt least squares
#
#p0s=res.x

print("Start parameters:",p0s)
print('Least squares, 2-point jac')
res = least_squares(pade_53_lsq, p0s, method='trf', jac='2-point', 
                    args=(ks, k2s, ls),  max_nfev=5000)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Start parameters: [2.3919, 0.2964, 1.3187, 1.3736, 0.29655, 0.4543, 0.5078, 1]
Least squares, 2-point jac
njev: 986
cost: 1.0404935673740506e-09
grad: [-3.68563660e-08 -1.87475545e-08 -1.26978125e-08 -2.73876825e-09
 -9.91550368e-08 -1.31568086e-07  5.14696853e-08  6.37290085e-08]
message: `xtol` termination condition is satisfied.
success: True
x: [2.39268253 0.24355109 1.32306897 0.48813835 1.67345685 0.31170218
 2.20311935 0.83989086]
chi2 = 2.338e-11
Er=1.746993,  Gamma=0.313923


In [16]:
print('Least squares, trust-region-reflective (default) with analytic jac')
res = least_squares(pade_53_lsq, p0s, method='trf', jac=pade_53j_lsq, 
                    args=(ks, k2s, ls),  max_nfev=5000)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, trust-region-reflective (default) with analytic jac
njev: 979
cost: 1.0404935461823376e-09
grad: [-2.17611480e-09 -1.09934897e-09 -7.30644619e-10 -1.61813359e-10
 -5.86444157e-09 -7.84455929e-09  3.03309240e-09  3.79793634e-09]
message: `gtol` termination condition is satisfied.
success: True
x: [2.39268253 0.24355147 1.32306903 0.48814693 1.67346062 0.31170181
 2.203125   0.83989187]
chi2 = 2.338e-11
Er=1.746993,  Gamma=0.313924


In [14]:
print('Least squares, Levenberg-Marquardt with analytic jac')
res = least_squares(pade_53_lsq, p0s, method='lm', jac=pade_53j_lsq, 
                    args=(ks, k2s, ls), max_nfev=5000)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, Levenberg-Marquardt with analytic jac
njev: 1377
cost: 1.040493546107574e-09
grad: [-3.05447952e-12 -1.55709745e-12 -1.05903234e-12 -2.27488606e-13
 -8.21444712e-12 -1.08594610e-11  4.27041673e-12  5.26040112e-12]
message: `gtol` termination condition is satisfied.
success: True
x: [2.39268253 0.24355148 1.32306903 0.4881469  1.67346084 0.3117018
 2.20312529 0.83989196]
chi2 = 2.338e-11
Er=1.746993,  Gamma=0.313924


In [17]:
print('Least squares, TRF with bounds')

bnds=([0,0,0, 0,0,0, 0,0],[np.inf,np.inf,np.inf, np.inf,np.inf,np.inf, np.inf,np.inf])

res = least_squares(pade_53_lsq, p0s, jac=pade_53j_lsq, bounds=bnds, 
                    args=(ks, k2s, ls),  max_nfev=5000)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, TRF with bounds
njev: 2427
cost: 1.0405087711138972e-09
grad: [-2.25103142e-09 -1.10836755e-09 -6.70515918e-10 -1.66634190e-10
 -6.11507222e-09 -8.44991906e-09  3.08251033e-09  4.05253810e-09]
message: `gtol` termination condition is satisfied.
success: True
x: [2.39268239 0.24352545 1.3230728  0.48819407 1.67292887 0.3117231
 2.202417   0.83967531]
chi2 = 2.338e-11
Er=1.747005,  Gamma=0.313857


In [ ]:
l0, a, b, d = res.x